In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

def criterion(out, label):
    return (label - out)**2


data = [(1.0,3.0), (2.0,6.0), (3.0,9.0), (4.0,12.0), (5.0,15.0), (6.0,18.0)]

W = torch.tensor([1.0], requires_grad=True)


X,label = data[1]

Y = X*W
loss = criterion(Y,label) 
loss.backward()

## d(loss)/dw = 2*(T-X*W)*(-X)

W.grad

In [ ]:
### Training with manually updating W with "Backward" ###

import torch
#from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

def criterion(out, label):
    return (label - out)**2


data = [(1.0,3.0), (2.0,6.0), (3.0,9.0), (4.0,12.0), (5.0,15.0), (6.0,18.0)]

W = torch.tensor([1.0], requires_grad=True)

lr = 0.01
temp = torch.tensor([0.0])


for epoch in range(20):
    for i, current_data in enumerate(data):
        W = torch.randn(1) # This line is for clean start
        W = temp
        W.requires_grad = True
        X, Y = current_data
        outputs = X*W
        loss = criterion(outputs, Y)
        loss.backward()
        W = W - lr* W.grad
        temp = W.detach()    # Not to deliver the grad, but just value
        if (i % 10 == 0):
            print("Epoch {} - loss: {}".format(epoch, loss))


### Test the trained network ###            
for i, current_data in enumerate(data):
    X, Y = current_data 
    outputs = X*W  
    print("when x = {}, y = {}".format(X, outputs))

In [ ]:
### Training with fancier version ###

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1,1,bias=False)  # in dim, out dim
    def forward(self, x):
        x = self.fc1(x)
        return x

net = Net()

print(net)
print(list(net.parameters()))

#input = torch.randn(1)
#out = net(input)

#def criterion(out, label):
#    return (label - out)**2
criterion = nn.MSELoss()


optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.5)
#optimizer = optim.Adam(net.parameters(), lr=0.005)


data = [(1.0,3.0), (2.0,6.0), (3.0,9.0), (4.0,12.0), (5.0,15.0), (6.0,18.0)]

for epoch in range(20):
    for i, current_data in enumerate(data):
        X, Y = current_data
        X, Y = torch.FloatTensor([X]), torch.FloatTensor([Y])
        optimizer.zero_grad()   
        outputs = net(X)
        loss = criterion(outputs, Y)
        loss.backward()
        optimizer.step()    ## This line is equivalent to "W = W - lr* W.grad"
        if (i % 10 == 0):
            print("Epoch {} - loss: {}".format(epoch, loss))

### Test the trained network ###            
for i, current_data in enumerate(data):
    X, Y = current_data
    X, Y = torch.FloatTensor([X]), torch.FloatTensor([Y])  
    out = net(torch.FloatTensor(X))  
    print("when x = {}, y = {}".format(X, out))
    